# RAG avec Mistral (Retrieval-Augmented Generation)
Fait par les Etudiants du Matser 2 :
- Mohamed DIALLO
- Yaya SANAOGO

**FST-USTTB**

Ce système est conçu pour extraire des informations pertinentes d'un document PDF et générer des réponses à des questions basées sur ces informations.

## Import des packages

In [4]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
import os
import pickle

## Copie du model Mistral 

**Chargement de la copie mistral en local (Ollama) :**

Pour des certaines raisons nous avons opter pour faire une copie du model mistral en local (ModelFile) a quelle comme temperature du 1 pour cette copie

*sources* : 
- [Running models with Ollama step-by-step](https://medium.com/@gabrielrodewald/running-models-with-ollama-step-by-step-60b6f6125807) 

- [ollama_custom_model](https://unmesh.dev/post/ollama_custom_model/)

- [How to Customize LLMs with Ollama](https://medium.com/@sumudithalanz/unlocking-the-power-of-large-language-models-a-guide-to-customization-with-ollama-6c0da1e756d9)

In [5]:
# Initialize model mistral
model = Ollama(model="mistral_copy")

## Chargement et Prétraitement du Document

1. **Chargement du Document PDF avec PyMuPDFLoader:**

Il s'agit de l'option d'analyse PDF la plus rapide. Elle contient des métadonnées détaillées sur le PDF et ses pages, et renvoie un document par page.

source : [Using PyMuPDF](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pymupdf)

Le document PDF est chargé à l'aide de PyMuPDFLoader depuis le chemin : /home/mohamed/Documents/Mohamed/2022-13_compressed.pdf.

In [6]:
pdf_path = "/home/mohamed/Downloads/2022-13_compressed.pdf"
loader = PyMuPDFLoader(pdf_path)
doc = loader.load()

In [7]:
# Questions sans RAG
question_1 = "Que pense la population malienne sur le retrait des forces Barkhane du Mali ?"
question_2 = "Quelles sont les actions prioritaires pour le gouvernement de transition selon les Maliens et quels sont les pourcentages de votes ?"

In [8]:
response_1 = model.invoke(question_1)

print(f"Question: \n{question_1}")
print(f"Réponse du LLM seul:\n{response_1}\n")

Question: 
Que pense la population sur le retrait des forces Barkhane du Mali ?
Réponse du LLM seul:
 En France, il n'y a pas de consensus clair sur le retrait des forces Barkhane du Mali. Certains soutiennent l'action militaire contre le terrorisme en Afrique de l'Ouest et considèrent que le retrait serait irresponsable, alors que d'autres considèrent qu'il s'agit d'une intervention étrangère non souhaitée et trop coûteuse. Il y a également des inquiétudes concernant les conséquences économiques et sécuritaires pour la région en cas de retrait des forces Barkhane. Une pollution par les munitions et des accroissements d'activité terroriste sont notamment cités comme préoccupations.



In [9]:
response_2 = model.invoke(question_2)

print(f"Question: \n{question_2}")
print(f"Réponse du LLM seul:\n{response_2}\n")

Question: 
Quelles sont les actions prioritaires pour le gouvernement de transition selon les Maliens et quels sont les pourcentages de votes ?
Réponse du LLM seul:
 Les résultats d'un sondage réalisé par l'Institut National de la Statistique (INStat) du Mali ont révélé les priorités des Maliens pour le gouvernement de transition. Voici les principales actions identifiées :

1. La lutte contre la corruption : 58,2 % des personnes interrogées pensaient que c'était une priorité urgente (hautement important ou très importante).
2. L'amélioration de la sécurité publique et la lutte contre le terrorisme : 43,9 % des personnes considéraient cette action comme prioritaire.
3. La réforme économique : 41,8 % des Maliens pensaient que cela était nécessaire.
4. La lutte contre la pauvreté : 35,6 % des personnes considéraient cette mesure comme une priorité urgente.
5. La lutte contre la famine : 29,8 % des Maliens pensaient que cela était important ou très important.
6. L'amélioration de l'enseig

2. **Découpage du Texte :**

Pour faciliter l'indexation et la recherche, le texte du PDF est découpé en morceaux plus petits. Ce découpage est réalisé avec RecursiveCharacterTextSplitter, utilisant des séparateurs variés (sauts de ligne, espaces, ponctuations). Chaque morceau de texte a une taille maximale de 300 caractères avec un chevauchement de 100 caractères pour conserver le contexte entre les morceaux.

LangChain propose différents types de séparateurs de texte suivant le tableau recapitulatif ci-dessous (premier lien dans le block des sources)
| Nom                               | Classes                                      | Sépare sur                     | Ajoute des métadonnées | Description                                                                                                                                                                                                                       |
|-----------------------------------|----------------------------------------------|-------------------------------|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Récursif                          | RecursiveCharacterTextSplitter, RecursiveJsonSplitter | Une liste de caractères définis par l'utilisateur |                        | Sépare le texte de manière récursive. Cette méthode essaye de garder les morceaux de texte liés les uns aux autres. C'est la méthode recommandée pour commencer à séparer du texte.                                                |
| HTML                              | HTMLHeaderTextSplitter, HTMLSectionSplitter  | Caractères spécifiques à HTML | ✅                     | Sépare le texte en fonction des caractères spécifiques à HTML. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le HTML).                                                                |
| Markdown                          | MarkdownHeaderTextSplitter                   | Caractères spécifiques à Markdown | ✅                     | Sépare le texte en fonction des caractères spécifiques à Markdown. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le Markdown).                                                        |
| Code                              | plusieurs langages                           | Caractères spécifiques au code (Python, JS) |                        | Sépare le texte en fonction des caractères spécifiques aux langages de programmation. 15 langues différentes sont disponibles au choix.                                                                                          |
| Token                             | plusieurs classes                            | Tokens                        |                        | Sépare le texte en tokens. Il existe plusieurs manières de mesurer les tokens.                                                                                                                                                    |
| Caractère                         | CharacterTextSplitter                        | Un caractère défini par l'utilisateur |                        | Sépare le texte en fonction d'un caractère défini par l'utilisateur. C'est l'une des méthodes les plus simples.                                                                                                                   |
| [Expérimental] Segmenteur Sémantique | SemanticChunker                             | Phrases                       |                        | Sépare d'abord en phrases. Puis combine celles qui sont adjacentes si elles sont suffisamment similaires sémantiquement. Tiré de Greg Kamradt.                                                                                     |
| AI21 Segmenteur de Texte Sémantique | AI21SemanticTextSplitter                    |                             | ✅                       | Identifie des sujets distincts qui forment des morceaux de texte cohérents et les sépare en conséquence.                                                                                                                           |


sources :

- [Types of Text Splitters](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/#types-of-text-splitters) 

- [Splitting text from languages without word boundaries](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries)

In [10]:
# Définir les séparateurs pour le découpage du texte
separators = [
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
]

In [11]:
# Initialiser le découpeur de texte avec les paramètres spécifiés
text_splitter = RecursiveCharacterTextSplitter(
    separators=separators,
    chunk_size=300, # Taille de chaque chunk en caractères
    chunk_overlap=100, # Chevauchement entre les chunks consécutifs
    length_function=len, # Fonction pour calculer la longueur du texte
    add_start_index=True, # Ajouter l'index de début à chaque chunk
)

In [12]:

chunks = text_splitter.split_documents(doc)
print(f"Split {len(doc)} documents into {len(chunks)} chunks.")

Split 121 documents into 583 chunks.


In [13]:
# Afficher un exemple de contenu de page et de métadonnées pour un chunk
page = chunks[0]
print(page.page_content)
print(page.metadata)

Enquête d’opinion «Que pensent les Malien(ne)s ?»
1
Chèr(e)s ami(e)s de la Friedrich-Ebert-Stiftung 
au Mali,  
 
Depuis 10 ans, nous demandons "Qu'en 
pensent les Malien(ne)s ? Lorsque nous avons 
commencé le Mali-Mètre en 2012, nous 
n'avions pas conscience de la portée qu'il
{'source': '/home/mohamed/Downloads/2022-13_compressed.pdf', 'file_path': '/home/mohamed/Downloads/2022-13_compressed.pdf', 'page': 2, 'total_pages': 121, 'format': 'PDF 1.4', 'title': 'Mise en page 1', 'author': 'IMPRIM COLOR', 'subject': '', 'keywords': '', 'creator': 'QuarkXPress(R) 14.21', 'producer': 'iLovePDF', 'creationDate': 'D:20220512192754Z', 'modDate': 'D:20240726083929Z', 'trapped': '', 'start_index': 0}


## Création et Gestion des Embeddings

1. **Génération des Embeddings :**

Les embeddings, qui sont des représentations vectorielles des morceaux de texte, sont générés à l'aide du modèle sentence-transformers/all-MiniLM-L6-v2 fourni par HuggingFaceEmbeddings.

Pourquoi le modèle sentence-transformers/all-MiniLM-L6-v2 alors qu'on a passé beaucoup de temps à télécharger Mistral et là, on vient nous parler d'un autre modèle voilà la question que l'on c'était poser au tout debut mais en des relectures et quelque recherche on as su que le sentence-transformers c'était juste pour nous aider dans la construction de notre base vectorielle il y en plein d'autre le plus connu reste le all-MiniLM-l6-V2 enfin disons par défaut. 

*sources :*

- [Embedding models](https://ollama.com/blog/embedding-models)

- [Using Langchain, Chroma, and GPT for document-based retrieval-augmented generation](https://developer.dataiku.com/12/tutorials/machine-learning/genai/nlp/gpt-lc-chroma-rag/index.html)

In [14]:
# Récupérer la fonction d'embeddings à partir des ressources du code env
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
2024-08-01 18:44:07.872556: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 18:44:08.048471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 18:44:08.195635: E e

2. **Cache des Embeddings :**

Pour optimiser les performances, les embeddings sont soit chargés depuis un cache (emb_cache.pkl), soit générés et ensuite enregistrés pour une utilisation future si le cache n'existe pas.

In [15]:
# Chemin vers le répertoire pour enregistrer la base de données Chroma
CHROMA_PATH = "embdb"
EMB_CACHE_PATH = "emb_cache.pkl"

In [16]:
# Charger les embeddings depuis le cache ou les générer si le cache n'existe pas
if os.path.exists(EMB_CACHE_PATH):
    with open(EMB_CACHE_PATH, 'rb') as f:
        cached_embeddings = pickle.load(f)
else:
    # Générer les embeddings pour chaque chunk
    cached_embeddings = embeddings.embed_documents([chunk.page_content for chunk in chunks])
    with open(EMB_CACHE_PATH, 'wb') as f:
        pickle.dump(cached_embeddings, f)

## Indexation et Recherche

1. **Base de Données Chroma :**

Les morceaux de texte, accompagnés de leurs embeddings, sont stockés dans une base de données Chroma. 
Cette base de données est persistée dans le répertoire embdb pour assurer une sauvegarde sur le disque.

In [17]:
# Créer une nouvelle base de données Chroma et ajouter les documents
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
db.add_documents(documents=chunks)

/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


['b1dfbf39-afec-4748-bdfb-284c30f17d62',
 'ea291443-70d7-46f1-bf66-7d50fa98e8d8',
 'cf946d6e-bbe5-4252-ae4b-37068a44cdaf',
 'cf88a59d-93bc-476c-b86e-6deaf5589b0d',
 '047d1a17-90f0-40f6-814b-60556bdf70e0',
 '80480d72-513e-413b-918b-627fd79575fa',
 '628ff8ef-529a-4b75-990b-2494a771308f',
 'bca9759b-1a25-49d6-90cc-557e26ba3bc7',
 '607e9a80-9f58-4c63-ad31-df4e02c33839',
 '10108710-95c6-4edb-8798-6500ba054432',
 '03874ed5-4db0-4d96-89b5-ff2bdc809849',
 '2986cd37-4d57-4cc0-9332-99aa3881f367',
 '9241ab1b-ce68-4047-8e8f-e0cbafbc9c22',
 'cd0995ac-90b4-4044-bff3-1c2b965d5a7c',
 '7151c3f7-35bc-40cf-9a1b-1c436ed5e762',
 '869d6ada-da1c-4b23-b49c-ca523b85ca3c',
 '02749d51-e234-45f8-9e02-e6c160d8f361',
 'b11d2aee-5771-4fd3-af1e-b191eff11dab',
 '2c69e8c0-390d-4fbe-b32e-70e35b3ed146',
 '6e80ef1f-484e-498a-b476-9dd1b8d6b44d',
 '12809666-b357-4a64-9451-221e87b4a6c7',
 '2809fe3f-212a-41ca-ae5a-1b8ad50aad64',
 'c53ed68b-670c-4915-b863-f0729fc2c1d3',
 'ea2b5bfe-8d83-4430-9263-dc1c3e6d039a',
 '5e08c16a-db84-

In [18]:
# Persister la base de données sur le disque
db.persist()
print(f"Enregistrement de {len(chunks)} chunks dans {CHROMA_PATH}.")

Enregistrement de 583 chunks dans embdb.


/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


2. **Modèle de Prompt :**

Un modèle de prompt est défini pour structurer la réponse. Le prompt intègre le contexte extrait des documents pertinents pour guider le modèle Mistral dans la génération de réponses claires et utiles.

In [39]:
# Création du promptTemplate
prompt_template = ChatPromptTemplate.from_template(
    """
        Tu t'appelles Okka.
        Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations. 
        Ton objectif est de fournir des réponses claires et utiles.

        L'utilisateur a posé la question suivante : "{question}"
        Recherche des documents pertinents dans la base de connaissances et utilise ces informations pour répondre à la question.

        Réponds à la question en utilisant uniquement le contexte suivant :
        {context}
        - -
        Réponds à la question ci-dessous en te basant uniquement sur le contexte fourni, dans la même langue que la question :
        Question : {question}
    """
)

3. **Recherche par Similarité et Ranking:**

Lorsqu'une question est posée, le système effectue une recherche par similarité dans la base de données Chroma.
Les documents les plus pertinents sont récupérés en fonction de leur similarité avec la question avec mécanisme de ranking.

Utilisons la première question pour faire ce travail dans un premier temps ensuite, nous créerons une fonction explicite pour faire l'ensemble du travail à savoir les recherches de similarité, le re-ranking etc.

In [20]:
# Extraire le contexte de la base de données en utilisant la recherche par similarité
results = db.similarity_search_with_relevance_scores(question_1, k=3)  # Récupérer plus de résultats pour le ranking
results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
top_results = results[:3]  # Prendre les 3 meilleurs résultats

context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])

4. **Génération de Réponse :**

La question posée est intégrée dans le modèle de prompt avec le contexte compressé, et la réponse est générée par le modèle Mistral.

In [21]:
# Générer une réponse basé sur le prompt
prompt = prompt_template.format(context=context_text, question=question_1)

rag_response_1 = model.invoke(prompt_template.format(context=context_text, question=question_1))
#response_text = model.invoke(prompt_template.format(compressed_prompt=compressed_prompt))

# Obtenir les sources des documents pertinents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Formater la réponse
formatted_response_1 = f"Response: {rag_response_1}\nSources: {sources}"
print(formatted_response_1)

Response:  La population interroquée considère qu'un impact positif sera résultant du retrait de la force Barkhane du Mali en termes de sécurité et lutte contre le terrorisme.
Sources: ['/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf']


**Note :** 

Pour raison de réutilisabilité du code, créons une fonction ask_rag qui va prendre en paramètre la query (question) fais les recherche de similarité, le re-ranking etc. et nous donne la réponse

In [22]:
def ask_rag(query):

  # Extraire le contexte de la base de données en utilisant la recherche par similarité
  results = db.similarity_search_with_relevance_scores(query, k=3)  # Récupérer plus de résultats pour le ranking
  results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
  top_results = results[:3]  # Prendre les 3 meilleurs résultats

  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])
  
  # Création du prompt avec le contexte
  prompt = prompt_template.format(context=context_text, question=query)
  
  # Générer une réponse basé sur le prompt
  response = model.invoke(prompt)
 
  # Obtenir les sources des documents pertinents
  sources = [doc.metadata.get("source", None) for doc, _score in top_results]

  # Formater la réponse
  formatted_response = f"Response: {response}\nSources: {sources}"
  print(formatted_response)
  
  return formatted_response, response

### Posons une nouvelle question (question_2):

In [23]:
formatted_response_2, rag_response_2 = ask_rag(question_2)

Response:  Les actions prioritaires pour le gouvernement de transition selon les Maliens comprennent :

1. Mise en œuvre d'une sécurité publique efficace pour garantir la protection des personnes et des biens dans tout le pays.
2. Établissement d'un plan économique national qui encourage l'investissement, soutient les petites entreprises et améliore la productivité agricole.
3. Réforme de l’administration publique pour augmenter sa transparence et réduire la corruption.
4. Augmentation de l'accès aux services d'éducation et de santé dans tout le pays, avec une émphasis sur l'amélioration des infrastructures scolaires et médicales.
5. Développement de projets pour améliorer la gestion de l’eau et l’assainissement dans les zones urbaines et rurales.
6. Mise en place d’une plateforme de dialogue avec toutes les parties prenantes, y compris les groupes civils, les organisations des femmes, les groupes de jeunes et les autorités locales pour récolter des idées et des suggestions sur la tran

### Posons une nouvelle question (question_3):

In [24]:
question_3 = "Quelles sont les principales réformes engagées et dans quelles domaines ?"
formatted_response_3, rag_response_3 = ask_rag(question_3)

Response:  Les trois principales réformes engagées sont dans le domaine de la sécurité. L'achat de matériels militaires et d'armements, l'augmentation des salaires et des primes ainsi que le renforcement de la formation sont les réformes énumérées par les personnes ayant entendu parler des réformes en cours.
Sources: ['/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf']


# Evaluation des réponses du RAG avec le BERTScore 

BERTScore est une mesure qui est apparue comme une alternative aux mesures d'évaluation traditionnelles dans le domaine du traitement du langage naturel (NLP). Elle est particulièrement utile pour évaluer la qualité du résumé de texte, en mesurant la similitude du résumé de texte avec le texte d'origine.

BERTScore utilise des embeddings contextuels pour évaluer la similarité sémantique entre les phrases, ce qui permet une évaluation plus fine que les méthodes basées sur la correspondance de n-grams, comme BLEU.

source : 

[BERTScore expliqué en 5 minutes](https://medium.com/@abonia/bertscore-explained-in-5-minutes-0b98553bfb71)

Ce article explique la motivation derrière BERTScore et détaille son architecture dont l'illustration est cidessous en image aussi
l'artilce explique les avantages et incovenients de l'utilisation de Bert score.

![Architecture](https://github.com/Mohameddiallo728/chatLLM/blob/main/Diagram-of-BERTScore-Retrieved-from-53.jpg?raw=true)

## Avantages :

- Sémantique : Évalue la similarité sémantique au lieu de se baser uniquement sur les mots exacts.

- Robustesse : Moins sensible aux variations lexicales et syntaxiques.

- Richesse Contextuelle : Tire parti des capacités de BERT à comprendre le contexte des phrases.

## Incovenients :

- Ressources : Nécessite plus de ressources computationnelles que les métriques traditionnelles.

- Complexité : Plus complexe à mettre en œuvre et à interpréter que les métriques basées sur des n-grams.

In [25]:
#!pip install bert-score

In [33]:
from bert_score import score

In [27]:
# Réponses générées par le RAG
rag_responses = [rag_response_1, rag_response_2, rag_response_3]

# Réponses de référence par humain
human_responses = [
    "Le retrait de la force Barkhane du Mali aura un impact positif sur l’économie de leur région pour (45 %) des personnes interrogées, et un impact négatif pour (9 %) d’entre elles. Toutefois, (36 %) de ces personnes pensent que la départ de Barkhane ne changera rien à l’économie dans leur région",
    "Les priorités majeures que les autorités de la transition devraient prendre en considération sont les suivantes : combattre l'insécurité (69 %), combattre l'insécurité alimentaire (35 %), créer de l'emploi (33 %), combattre la pauvreté (31 %), restaurer la paix et la stabilité (30 %) et progresser dans le système éducatif (23,8 %).",
    "De 2017 à 2022, les réformes majeures menées au Mali concernent principalement trois domaines : la paix et la sécurité, l'emploi des jeunes et la sécurité alimentaire, ainsi que la relance de l'économie."
]

In [35]:
# Calcul des scores BERTScore
P, R, F1 = score(rag_responses, human_responses, lang='fr')


Error while downloading from https://cdn-lfs.huggingface.co/bert-base-multilingual-cased/876f584f15ebf14887dec17539c114bb99a032e96b9e72507a51c41e205337fc?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1722799076&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjc5OTA3Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9iZXJ0LWJhc2UtbXVsdGlsaW5ndWFsLWNhc2VkLzg3NmY1ODRmMTVlYmYxNDg4N2RlYzE3NTM5YzExNGJiOTlhMDMyZTk2YjllNzI1MDdhNTFjNDFlMjA1MzM3ZmM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=nsw4BFt6Tz6wLGKMNiZBLQvtsRhgw6Z1Q71Jd9V2aqcYi62-KKecA%7EKcmlAbPnnD5aXVTASXhsrPNf6cARw7Zlb6n7pTcF%7EAXCU4C-eIgVVKnGT92vO2UV-jSk9mFELgatsOmyfmJHl9pgzdAnPEaR8METFAfsw9e8BaXhtGDuuiQgKHJ0lxg4GAAk1lunQ4Alth8ZHFJowUFZtpuzx%7EvIgf-8cgOTf9NuRvXPXX13VAMXTXctUl%7ErQ--JoEwrpyej3GcRGo-6Ybg1eSqEuc9BusztJkDc5Uo0QBFlHgwGFEoTqo-OzQ%7E0vcAIxNByHJisiRG6IRg9GGNEiejvDWng__&Key

In [38]:
# Affichage des résultats
for i, (p, r, f1) in enumerate(zip(P, R, F1)):
    print(f"* Question {i+1}:")
    print(f" - Précision: {p:.3f}")
    print(f" - Rappel: {r:.3f}")
    print(f" - F1-score: {f1:.3f}")  

* Question 1:
 - Précision: 0.755
 - Rappel: 0.693
 - F1-score: 0.723
* Question 2:
 - Précision: 0.652
 - Rappel: 0.676
 - F1-score: 0.664
* Question 3:
 - Précision: 0.763
 - Rappel: 0.778
 - F1-score: 0.770


# Résultats

Le système RAG développé permet de traiter efficacement un document PDF et d'extraire des informations pertinentes pour répondre à des questions spécifiques. La réponse générée inclut également les sources des documents pour assurer la traçabilité des informations fournies.

L'evaluation des reponses generer avec le BERTScore montre :

*Question 1*

- Précision: 0.755 : La réponse est assez précise, avec 75.5% des mots générés correspondant aux mots attendus.
- Rappel: 0.693 : Le modèle a inclus environ 69.3% des informations importantes attendues dans la réponse.
- F1-score: 0.723 : La qualité globale de la réponse est bonne, équilibrant précision et rappel.


*Question 2*

- Précision: 0.652 : La réponse est moins précise, avec 65.2% des mots générés pertinents.
- Rappel: 0.676 : Le modèle a inclus environ 67.6% des informations importantes attendues.
- F1-score: 0.664 : La qualité globale est moyenne.

*Question 3*

- Précision: 0.763 : La réponse est précise, avec 76.3% des mots générés correspondant aux mots attendus.
- Rappel: 0.778 : Le modèle a inclus environ 77.8% des informations importantes attendues.
- F1-score: 0.770 : La qualité globale de la réponse est très bonne, bien équilibrée entre précision et rappel.